In [30]:
import requests
import pandas as pd


def get_odata(target_url: str, max_requests: int = 100) -> pd.DataFrame:
    """ This functions retrieves data from a given dataset from the Central Bureau of Statistics (CBS).
    The API returns the data in a JSON format, which is converted to a pandas DataFrame.

    Each data request contains a maximum of 10.000 cells, therefore in order to retrieve the whole dataset,
    the API is constantly called. 

    Args:
        target_url (str): The target url which points to a dataset
        max_requests (int): Indicates the maximum number of requests should be made

    Returns:
        pd.DataFrame: DataFrame that contains all retrieved data from CBS
    """
    # Creation of a DataFrame
    df: pd.DataFrame = pd.DataFrame()
    request_index: int = 1

    while target_url and (request_index <= max_requests):
        print(f"Starting request {str(request_index)}/{str(max_requests)}...")

        r = requests.get(target_url).json()
        df = pd.concat([df, pd.DataFrame(r['value'])], ignore_index=True)
        
        if '@odata.nextLink' in r:  
            target_url = r['@odata.nextLink']
        else:
            target_url = None
        
        request_index = request_index + 1
        
    return df

In [38]:
dataset_identifier: str = "83648NED"
table_url:str = f"https://odata4.cbs.nl/CBS/{dataset_identifier}"
target_url:str = f"{table_url}/Observations"
data: pd.DataFrame = get_odata(target_url, 3)

Starting request 1/3...
Starting request 2/3...
Starting request 3/3...


In [37]:
data_path: str = r"../data"


def dataframe_to_csv(df: pd.DataFrame, save_path: str, file_name: str) -> None:
    """ Converts a DataFrame to a csv file and saves it at a specific location

    Args:
        df (pd.DataFrame): DataFrame to be converted to CSV
        save_path (str): The folder in which the file needs to be saved
        file_name (str): The actual name of the file
    """
    df.to_csv(f"{save_path}/{file_name}", ",", index=False, encoding="utf-8")


dataframe_to_csv(data, data_path, "crime_data.csv")